random loops 
- completely random anchors
- completely random anchors that are within the chromosome
- only # of anchors and loops per tissue are preserved


- moved to `build_random_loops.py`
june 23, 2020


this is for debugging purposes


In [7]:
import pybedtools
import pandas as pd
import os,glob,sys
import numpy as np

In [6]:
save_dir = '../data/tissue_crms_random_manual'

def make_dir(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
make_dir(save_dir)

In [2]:
normal_tissues = ['Airway','Astrocytes','Bladder','Colon','Esophageal',#'GDSD0','GDSD3',
                  'GDSD6','GM12878','HMEC','Melanocytes','Ovarian',
'Pancreas','Prostate','Renal','Thyroid','Uterine']
normal_tissues_dict = dict(zip(normal_tissues,range(len(normal_tissues))))

In [8]:
# 1. set directories
merged_dir = '../data/interim/merged'
save_anchor_path_dir = os.path.join(merged_dir, 'anchors_random')
save_loop_path_dir = os.path.join(merged_dir, 'loops_random')
make_dir(save_anchor_path_dir)
make_dir(save_loop_path_dir)

anchor_path=os.path.join(merged_dir,'anchors')

In [10]:
# 2. get all anchors, counts for loops and anchors for each tissue 
anchors_all = set()
anchor_counts = {}
loop_counts = {}
for anchor_file in glob.glob(os.path.join(anchor_path, '*csv')):
    tissue = os.path.basename(anchor_file).split('.')[0]
    if tissue not in normal_tissues:
        continue
    print(tissue)
    loop_file = os.path.join(merged_dir, 'loops', tissue+'.loops.csv')
    tissue_anchor_df = pd.read_csv(anchor_file,index_col=0)
    tissue_loop_df = pd.read_csv(loop_file,index_col=0)

    num_anchors = tissue_anchor_df.shape[0]
    num_loops = tissue_loop_df.shape[0]
#     num_loops_counts = sum(tissue_loop_df['count'])
    anchor_counts[tissue] = num_anchors
    loop_counts[tissue] = num_loops
    
    anchors = set(tissue_anchor_df.anchors.values)
    anchors_all = anchors_all.union(anchors)

Colon
Uterine
Renal
Esophageal
Ovarian
Pancreas
Prostate
Bladder
GM12878
Melanocytes
Thyroid
HMEC
GDSD6
Astrocytes
Airway


In [11]:
display(tissue_anchor_df[:5])
display(tissue_loop_df[:5])

,anchors,chr,start,end,sample
0,chr10_100025000_100030000,chr10,100025000,100030000,Air_B1
1,chr10_100055000_100060000,chr10,100055000,100060000,Air_B1
2,chr10_100065000_100070000,chr10,100065000,100070000,Air_B1
3,chr10_101150000_101155000,chr10,101150000,101155000,Air_B1
4,chr10_101185000_101190000,chr10,101185000,101190000,Air_B1


,source,target,count
0,chr10_100020000_100025000,chr10_100055000_100060000,13
1,chr10_100020000_100025000,chr10_100065000_100070000,13
2,chr10_100025000_100030000,chr10_100055000_100060000,14
3,chr10_100025000_100030000,chr10_100065000_100070000,11
4,chr10_101150000_101155000,chr10_101180000_101185000,14


In [16]:
len(anchors_all)

281172

In [22]:
# 3. for each tissue make a random anchor and loop file
for tissue in ['Colon']:#anchor_counts.keys():
    random_anchors = np.random.choice(list(anchors_all), anchor_counts[tissue],replace=False)
    random_anchors_df = pd.DataFrame(columns = ['anchors','chr','start','end'])
    random_anchors_df['anchors'] = random_anchors 
    random_anchors_df[['chr','start','end']] = random_anchors_df['anchors'].str.split('_',expand=True)
    random_anchors_df.to_csv(os.path.join(save_anchor_path_dir, tissue+'.anchors.csv'))
    
    # make random loops
    target_arr = []
    for anchor in random_anchors:
        chrom = anchor.split('_')[0]
        target_choice = np.random.choice(self.chr_anchor_prob_dict[chrom].anchor, p=self.chr_anchor_prob_dict[chrom].prob)
        if not self.allow_self_loop:
            while target_choice==anchor:
                target_choice = np.random.choice(self.chr_anchor_prob_dict[chrom].anchor, p=self.chr_anchor_prob_dict[chrom].prob)

        target_arr.append(target_choice)

In [23]:
random_anchors_df

,anchors,chr,start,end
0,chr1_116850000_116855000,chr1,116850000,116855000
1,chr17_1670000_1675000,chr17,1670000,1675000
2,chr2_199760000_199765000,chr2,199760000,199765000
3,chr4_151785000_151790000,chr4,151785000,151790000
4,chr14_76935000_76940000,chr14,76935000,76940000
...,...,...,...,...
101554,chr3_194485000_194490000,chr3,194485000,194490000
101555,chr4_154065000_154070000,chr4,154065000,154070000
101556,chr1_16550000_16555000,chr1,16550000,16555000
101557,chr18_43660000_43665000,chr18,43660000,43665000


In [20]:
random_anchors_df['anchors'].str.split('_',expand=True)

,0,1,2
0,chr22,32055000,32060000
1,chr15,74535000,74540000
2,chr7,45110000,45115000
3,chr10,99590000,99595000
4,chr1,43135000,43140000
...,...,...,...
101554,chr3,195990000,195995000
101555,chr12,14175000,14180000
101556,chr10,10790000,10795000
101557,chr13,78225000,78230000


In [27]:
# 
os.path.join(interim_filepath,'merged/anchors_random_bed_sort')pybedtools.BedTool.from_dataframe(random_anchors_df).to_dataframe()#sort()


,chrom,start,end,name
0,chr1,116850000,116855000,chr1_116850000_116855000
1,chr17,1670000,1675000,chr17_1670000_1675000
2,chr2,199760000,199765000,chr2_199760000_199765000
3,chr4,151785000,151790000,chr4_151785000_151790000
4,chr14,76935000,76940000,chr14_76935000_76940000
...,...,...,...,...
101554,chr3,194485000,194490000,chr3_194485000_194490000
101555,chr4,154065000,154070000,chr4_154065000_154070000
101556,chr1,16550000,16555000,chr1_16550000_16555000
101557,chr18,43660000,43665000,chr18_43660000_43665000


In [ ]:
os.path.join('../data/interim','merged/anchors_random_bed_sort')